# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
### Disciplina CD004 - Metodologia de Aprendizado de Máquina Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---


<br>

## **Aula 07** - **Tópico: Interpretação de Modelos Preditivos**

<br>

**Objetivo deste notebook**: Exemplificar o uso de diferentes estratégias para interpretar ou explicar modelos preditivos treinados com algoritmos de aprendizado de máquina. Discutiremos relevância de atributos (estimada por árvores ou por permutação), Partial Dependence Plots, e SHAP.
<br>

---





##**Predição de 'churn' de clientes de serviço de telecomunicação**

Os dados que utilizaremos neste notebook se referem a uma empresa de telecomunicações fictícia que forneceu serviços de telefone residencial e Internet para clientes na Califórnia. O conjunto de dados possui informações sobre os serviços para os quais cada cliente se inscreveu (telefone, várias linhas, internet, segurança online, backup online, proteção de dispositivos, suporte técnico e streaming de TV e filmes), informações da conta do cliente (há quanto tempo eles são clientes, contrato, forma de pagamento, cobrança sem papel, cobranças mensais e cobranças totais) e informações demográficas sobre os clientes (sexo, faixa etária, se eles têm parceiros e dependentes). Há, ainda, uma coluna chamada 'churn', que indica os clientes que desistiram do contrato do serviço no último mês. O objetivo da tarefa preditiva é identificar o churn (i.e., saída) de clientes a partir das informações coletadas. Os dados podem ser acessados neste [link](https://www.kaggle.com/datasets/blastchar/telco-customer-churn).




---



In [ ]:
## Carregando as bibliotecas básicas necessárias
# A primeira linha é incluída para gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline              
import pandas as pd             # para análise de dados 
import matplotlib.pyplot as plt # para visualização de informações
import seaborn as sns           # para visualização de informações
import numpy as np              # para operações com arrays multidimensionais
sns.set()

###Carregando os dados

Nas células a seguir, vamos repetir o procedimento padrão de carregar os dados e fazer algumas modificações e preparação básica dos dados para a tarefa de aprendizado supervisionado.

In [ ]:
## Carregando os dados
df = pd.read_csv("https://drive.google.com/uc?export=view&id=10VrzI8mA2wPvkNIDaLzv-IglPU-Febtf")#,na_values="NA")
df  

In [ ]:
## Características gerais do dataset
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

A coluna *'Churn'* contém a classificação de cada instância. Vamos avaliar a distribuição de classes do problema.

In [ ]:
## Distribuição do atributo alvo
plt.hist(df['Churn'])
plt.title("Distribuição do atributo alvo")
plt.show()

In [ ]:
df.info()

O atributo customerID é único para cada instância e no geral não possui valor preditivo. Iremos removê-lo da análise.

In [ ]:
customer_ids = df['customerID']
df = df.drop(['customerID'], axis=1)

In [ ]:
##remove as duplicatas
df=df.drop_duplicates(keep='last')

Como atributos categóricos e numéricos podem demandar pré-processamentos diferentes, vamos separar os respectivos "nomes" em dois vetores distintos, facilitando a manipulação dos dados posteriormente.

In [ ]:
## Separa os atributos em vetores, de acordo com o tipo de dado (categórico ou numérico)
cat_columns=list(df.drop(['Churn'], axis=1).select_dtypes(include=["object"]).columns)
print(cat_columns)

num_columns=list(df.select_dtypes(include=["int64", "float64"]).columns)
print(num_columns)

Antes de iniciar o treinamento do modelo, lembre-se que é recomendado sempre reservar uma porção dos dados para teste, a qual somente será utilizada para avaliação do modelo final (após todo o processo de treinamento e otimização de hiperparâmetros).

Vamos fazer esta divisão, separando 20% para teste. Entretanto, primeiro precisamos dividir os dados entre atributos (X) e classe (y). Também iremos codificar as classes Yes/No para 1/0.

In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = df.drop(['Churn'], axis=1)
y = df['Churn'].values

Faremos o mapeamento das classes Yes/No para 1/0. Por padrão, as funções de avaliação assumem que a classe 1 é a positiva/de interesse.

In [ ]:
## substitui No' por 0, 'Yes' por 1
y = np.array([0 if y=='No' else 1 for y in y]) 

In [ ]:
from sklearn.model_selection import train_test_split
## Faz a divisão entre treino (80%) e teste (20%).
## O conjunto de treino representa os dados que serão usados
## ao longo do desenvolvimento do modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=42) 

---


### Pipeline de pré-processamento e treinamento

Continuaremos usando o conceito de Pipelines para realizar o pré-processamento de atributos e o treinamento do modelo. Serão aplicados os seguintes passos, conforme notebooks anteriores:

*   Atributos **numéricos**: imputação de valores faltantes (com o método KNN) e normalização
*   Atributos **categóricos**: imputação de valores faltantes (com a moda), one-hot encoding, e normalização.

A última etapa do pipeline é o treinamento do modelo preditivo. Neste notebook, vamos usar o classificador Random Forest. A estratégia utilizada para lidar com desbalanceamento de dados será um aprendizado sensível a custo, através do hiperparâmetro class_weight.

**Observação:** Salienta-se que o objetivo deste notebook é entender o funcionamento dos métodos de interpretação de modelos. Esta etapa é normalmente feita ao final de todo o processo de desenvolvimento de modelos preditivos (incluindo a otimização de hiperparâmetros com estratégias como nested cross-validation). Neste notebook vamos simplificar essa análise, usando um único algoritmo e um processo bem rápido de otimização de hiperparâmetros com validação cruzada.

In [ ]:
## Nesta célula definimos o pipeline e fazemos o ajuste aos dados

from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)


## pipeline específico para os atributos numéricos
num_pipeline = Pipeline(steps=[
                         ('imputer', KNNImputer(weights="uniform"))])

## pipeline específico para os atributos categóricos
cat_pipeline = Pipeline(steps=[
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

## ColumnTransformer para aplicar cada pipeline ao respectivo tipo de atributo
data_pipeline = ColumnTransformer(transformers=[
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])

## define pipeline que une as transformações definidas anteriormente e aplica a 
## normalização em todos os atributos
prep_pipeline = Pipeline(steps=[
                 ('data_transform', data_pipeline),
                 ('data_normalize',MinMaxScaler())])

## finalmente, pipeline que executa o pré-processamento dos dados seguido pelo
## treinamento do modelo floresta aleatória.
tree_pipeline = Pipeline(steps=[
                          ('data_preproc', prep_pipeline), 
                          ('clf',RandomForestClassifier(random_state=42,n_estimators=50,class_weight='balanced_subsample'))])

## define uma grid de hiperparâmetros a serem testados e avaliados
tree_param_grid = {
    'clf__max_depth': [4, 5, 6, 7, 8]
}

## define e aplica uma grid search. 
tree_grid_search = GridSearchCV(
    estimator=tree_pipeline,
    param_grid=tree_param_grid,
    scoring='f1',
    cv=cv,
    refit=True
)

tree_grid_search = tree_grid_search.fit(X=X_train, y=y_train)


In [ ]:
print(tree_grid_search.best_params_)

Cria uma versão dos dados pré-processados, de acordo com o pipeline definido acima (pre_proc, sem a etapa de classificação)

In [ ]:
X_train_prep = tree_pipeline['data_preproc'].fit_transform(X_train)
X_test_prep = tree_pipeline['data_preproc'].transform(X_test)
columns = np.append(num_columns,prep_pipeline[0].named_transformers_['categorical']['encoder'].get_feature_names_out(cat_columns))
df_train_prep = pd.DataFrame(X_train_prep, columns=columns)
df_test_prep = pd.DataFrame(X_test_prep, columns=columns)

Iniciamos fazendo uma avaliação do modelo. Para interpretação dos modelos, é sempre interessante que o desempenho quanto a sua capacidade preditiva seja a melhor possível. Nosso modelo possui um recall relativamente bom, mas baixa precisão. Não aprofundaremos a investigação nesse sentido, mas é importante sempre lembrar que a interpretação de modelos é mais produtiva e relevante quando o modelo apresenta potencial de solução da tarefa investigada.

In [ ]:
y_test_pred = tree_grid_search.predict(X=X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score,accuracy_score,f1_score,ConfusionMatrixDisplay ## para avaliação dos modelos

cm = confusion_matrix(y_test, y_test_pred,labels=tree_grid_search.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=tree_grid_search.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y_test, y_test_pred),3)))
print('Recall: {}'.format(round(recall_score(y_test, y_test_pred,pos_label=1),3)))
print('Precisão: {}'.format(round(precision_score(y_test, y_test_pred,pos_label=1),3)))
print('F1-Score: {}'.format(round(f1_score(y_test, y_test_pred,pos_label=1),3)))

### Análise de importância de atributos

Como vimos na Aula 06, alguns modelos são capazes de estimar uma importância para cada atributo a partir dos dados e do processo de treinamento. Florestas Aleatórias é um algoritmo que possui essa característica, retornando dados úteis para iniciar a interpretação do modelo.


In [ ]:
feature_importance = tree_grid_search.best_estimator_['clf'].feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 8))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(columns)[sorted_idx])
plt.title('Feature Importance')


Enquanto o gráfico acima demonstra a importância média (entre o ensemble de árvores), também é possível avalair como a importância muda entre as N árvores do ensemble.

In [ ]:
importances = tree_grid_search.best_estimator_['clf'].feature_importances_
std = np.std([tree.feature_importances_ for tree in tree_grid_search.best_estimator_['clf']], axis=0)

In [ ]:
forest_importances = pd.Series(tree_grid_search.best_estimator_['clf'].feature_importances_, index=columns)

fig, ax = plt.subplots(figsize=(12, 6))
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")

A análise de importância de atributos pode ser aplicada a qualquer modelo a partir da estratégia de importância baseada em permutação: a importância de um atributo é estimada como o nível de redução no score de um modelo (onde score é a métrica de desempenho sendo otimizada) ao se permutar aleatoriamente os valores deste atributo.

Abaixo vamos aplicar esse método para a Floresta Aleatória. Ele é chamado um método 'agnóstico', pois pode ser usada com qualquer modelo derivado de aprendizado supervisionado. O scikit-learn fornece o método permutation_importance.

In [ ]:
from sklearn.inspection import permutation_importance

## faz permutações, treina e avalia modelos, e estima diferença de desempenho (proxy importância)
perm_importance = permutation_importance(tree_grid_search.best_estimator_['clf'], X_train_prep, y_train, n_repeats=30,random_state=0)

Analisando a estimativa de importância para cada atributo.

In [ ]:
sorted_idx = perm_importance.importances_mean.argsort()
fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(sorted_idx)), perm_importance.importances_mean[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(columns)[sorted_idx])
plt.title('Permutation Importance')

É interessante percebermos que enquanto a relevância de atributos pode nos ajudar a entender quais atributos mais impactam no modelo, ela não nos informa como ele impacta, isto é, se aumentar ou diminuir o valor do atributo aumenta ou diminui as chances para uma determinada classe. 

### Partial dependence plot (PDP)

Analisa o efeito marginal que um (ou dois) atributos têm sobre o resultado previsto de um modelo de aprendizado de máquina. Para um determinado valor do atributo analisado, representa a previsão média do modelo para o caso em que forçamos todas as instâncias a assumirem aquele valor. Permite analisar uma relação entre atributo e saída no modelo.

Nas células a seguir, vamos explorar PDP para visualizar a forma como alguns atributos impactam na decisão do modelo.

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

features_to_display = ['Contract_Two year','Contract_One year','Contract_Month-to-month','TotalCharges','MonthlyCharges', 'tenure']

fig, ax = plt.subplots(figsize=(15, 7))

display_tree = PartialDependenceDisplay.from_estimator(
       estimator=tree_grid_search.best_estimator_['clf'],
       X=df_train_prep,
       features = features_to_display,
       kind='average',
       subsample=50,
       n_jobs=3, 
       grid_resolution=20,
       random_state=42,
       ax=ax,      
)

fig.suptitle('PDP ', y=0.95);

In [ ]:
features_to_display = ['TechSupport_No internet service','DeviceProtection_No internet service','OnlineSecurity_No internet service',
                       'TechSupport_No','DeviceProtection_No','OnlineSecurity_No',
                       'TechSupport_Yes','DeviceProtection_Yes','OnlineSecurity_Yes']

fig, ax = plt.subplots(figsize=(15, 7))

display_tree = PartialDependenceDisplay.from_estimator(
       estimator=tree_grid_search.best_estimator_['clf'],
       X=df_train_prep,
       features = features_to_display,
       kind='average',
       subsample=50,
       n_jobs=3, 
       grid_resolution=20,
       random_state=42,
       ax=ax,      
)

fig.suptitle('PDP ', y=0.95);

In [ ]:
features_to_display = [('tenure','TotalCharges'),('MonthlyCharges','Contract_Month-to-month')]

fig, ax = plt.subplots(figsize=(10, 6))

display_tree = PartialDependenceDisplay.from_estimator(
       estimator=tree_grid_search.best_estimator_['clf'],
       X=df_train_prep,
       features = features_to_display,
       kind='average',
       subsample=50,
       n_jobs=3, 
       grid_resolution=20,
       random_state=42,
       ax=ax,      
)

fig.suptitle('PDP ', y=0.95);

### SHAP

O SHAP explica a previsão de uma instância x calculando a contribuição de cada atributo para a previsão. Valor de Shapley é a contribuição marginal média de um valor de atributo em todas as coalizões possíveis (isto é, combinação entre valores de atributos).

Utilizaremos a biblioteca shap.
Ler mais em: https://github.com/slundberg/shap

In [ ]:
##instalando a biblioteca
!pip install shap

In [ ]:
import shap
## como o processo de calcular valores SHAP é custoso, usamos apenas uma amostra dos dados
X_train_sample = shap.utils.sample(X_train_prep, 500)
# compute the SHAP values for the linear model
explainer = shap.Explainer(tree_grid_search.best_estimator_['clf'].predict, X_train_sample)
shap_values = explainer(X_test_prep)

In [ ]:
#shap.plots.bar(shap_values[0]) ## semelhante ao plots.waterfall.
shap.plots.waterfall(shap_values[0], max_display=14) ##analisa a primeira instância de teste (0)

In [ ]:
print(columns[32])

Para uma explicação global, o gráfico beeswarm é mais interessante. Os atributos são ordenados por relevância de acordo com análise SHAP e cada ponto representa uma instância de teste. A cor dos pontos e a sua posição em relação ao eixo x permitem verificar se um determinado atributo tende a contribuir ou não para predição de uma classe quando assume determinados valores. Por exemplo, o atributo 32, quando apresenta valores mais altos (nos nossos dados, valor de 1/máximo), contribuir para a predição da classe positiva ('churn').

In [ ]:
shap.plots.beeswarm(shap_values, max_display=20,feature_names=columns)

In [ ]:
shap.plots.bar(shap_values)